# Исследование надёжности заёмщиков

Заказчик — кредитный отдел банка.
Входные данные, полученные от банка — статистика о платёжеспособности клиентов. Таблица с данными включает в себя следующую информацию о каждом клиенте:
1. Количество детей
2. Возраст
3. Общий стаж работы
4. Сведения об образовании
5. Семейное положение
6. Пол
7. Тип занятости
8. Общий доход
9. Цель кредита
10. Факт наличия или отсутствия задолженностей по кредиту

**Задача** - исследовать зависимость факта погашения кредита в срок от следующих факторов:
1. Семейное положение
2. Количество детей
3. Уровень дохода
4. Цель кредита

## 1. Загрузка данных. Общая информация

In [2]:
import pandas as pd
data = pd.read_csv('C:/Users/leoci/Яндекс Практикум/Projects/Data_preprocessing/data.csv')

In [3]:
data.head(10)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
5,0,-926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,покупка жилья
6,0,-2879.202052,43,высшее,0,женат / замужем,0,F,компаньон,0,240525.971920,операции с жильем
7,0,-152.779569,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823.934197,образование
8,2,-6929.865299,35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856.832424,на проведение свадьбы
9,0,-2188.756445,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425.938277,покупка жилья для семьи


In [ ]:
print(data.info())
print(data.columns)

### Вывод

Таблица состоит из 12 столбцов и 21525 строк. 

Есть 2 столбца с пропущенными значениями: "общий трудовой стаж" и "ежемесячный доход", при чём, количество пропущенных значений в данных столбцах совпадает. 

Названия столбцов заданы корректно (нет пробелов, всё в нижнем регистре, латинским алфавитом) => с таблицей можно удобно работать.

## 2. Предобработка данных

### Обработка пропусков

In [ ]:
nan_data = data.loc[data['days_employed'].isna() == True].reset_index(drop = True)
print(nan_data.info())
print('')

data['days_employed'] = data['days_employed'].fillna(0)

median_income = data.groupby('income_type')['total_income'].median()
for row in range(len(data)):
    if data.loc[row, 'total_income'] != data.loc[row, 'total_income']:
        type = data.loc[row, 'income_type']
        data.loc[row, 'total_income'] = median_income[type]

print(data.info())


### Вывод

В Шаге 1 мы отметили, что количество пропущенных значений в столбцах "общий трудовой стаж" и "ежемесячный доход" совпадает. Проверим, совпадают ли пропуски в столбцах по строкам. Создаём таблицу, состоящую из данных с пропусками в столбце "общий трудовой стаж", смотрим сводную инфомацию по таблице и видим, что в столбце "ежемесячный доход" так же нет ни одного значения, значит пропущенные значения в столбцах "общий трудовой стаж" и "ежемесячный доход" совпадают по строкам. 

Вывод: у некторох клиентов, по каким-то причинам, отсутствуют данные о доходах и трудовом стаже. Причиной может быть нежелание клиента указывать данную информацию.

Количество строк с пропусками составляет 10% от общих данных. Достаточно большой процент, просто удалить пропуски нельзя, необходимо их заполнить. 

Рассмотрим столбец "Общий трудовой стаж". Для заполнения пропусков в нём хорошо подойдет среднее значение по возрастным группам, т.к. данные строго ограничены возрастом человека, а значит не могут иметь сильно выбивающихся значений. Однако, в этом столбце есть артефакты в виде отрицательных значений и значений стажа в несколько сотен лет. Это очевидные ошибки в данных, нужно преобразовать отрицательные значения в положительные, а многовековые трудовые стажи удалить и заполнить обратно вместе с осталбными пропусками. Общий трудовой стаж нас не интересует в рамках данной задачи, поэтому просто заполним пропуски в нём нулями.

Рассмотрим столбец "ежемесячный доход". Во-первых, в этом столбце некоторые данные могут сильно выбиваться, поэтому разумно заполнить пропуски медианным значением. Во-вторых, в данных присутствуют люди с разным типом занятости, поэтому сгруппируем доходы людей по типу занятости и заполним пропуски медианными значениями на основании полученных групп.

После обработки пропусков проверяем сводную информацию по таблице, убеждаемся что пропусков не осталось.

### Замена типа данных

In [ ]:
data['days_employed'] = data['days_employed'].astype('int')
data['total_income'] = data['total_income'] / 1000
data['total_income'] = data['total_income'].astype('int')
display(data.head())
print('')
print(data.info())

### Вывод

Рабочий стаж удобнее видеть в целочисленном формате. 

Среднегодовой доход для наглядности удобно представить в тысячах рублей, в целочисленном формате.

Все значения в столбцах имеют одинаковый тип данных, поэтому для замены типа данных применяем метод astype() к столбцу.

### Обработка дубликатов

In [ ]:
def lower(text):
    lower_text = text.lower()
    return lower_text

data['education'] = data['education'].apply(lower)
print(data.duplicated().sum())
print('')
data = data.drop_duplicates().reset_index(drop=True)
print(data.duplicated().sum())

### Вывод

В предоставленных данных нет ни одного столбца, каждое значение которого должно было бы быть уникальным, поэтому проверяем наличие дубликатов методом duplicated() с использованием метода sum(). Дубликаты в данных есть, возможно из-за технических ошибок, например дважды отправленной одной и той же анкеты. Удаляем их с восстановлением индексов в таблице методом drop_duplicates с применением метода reset_index, делаем контрольную проверку на наличие дубликатов.

### Лемматизация

In [ ]:
from collections import Counter
from pymystem3 import Mystem
m = Mystem()

def lemmas(purpose):
    purpose_lemma = m.lemmatize(purpose)
    return purpose_lemma
data_purpose_lemmas = data['purpose'].apply(lemmas)
print(data_purpose_lemmas.value_counts())
print('')

def purpose_id(purpose_lemma):
    if 'автомобиль' in purpose_lemma:
        return 0
    if 'свадьба' in purpose_lemma:
        return 1
    if 'недвижимость' in purpose_lemma:
        return 2
    if 'жилье' in purpose_lemma:
        return 2
    if 'образование' in purpose_lemma:
        return 3
data['purpose_id'] = data_purpose_lemmas.apply(purpose_id)




In [ ]:
data.head(10)

### Вывод

Создадим новый столбец, в котором будут содержаться лемматизированные значения целей кредита. Скопируем существующий столбец с целями кредитов, а дальше применим к нему лемматизацию. Выведем все существующие лемматизированные значения целей кредита. Видим, что множество совпадающих целей записаны по разному. Присвоим свой индекс для каждой цели, для этого создадим функцию, проверяющую лемматизированную цель на наличие ключевых слов: "жилье" или "недвижимость", "автомобиль", "образование", "свадьба" и возвращающую соответствующий индекс цели кредита.

### Категоризация данных

In [ ]:
print(data.columns)
print('')
data_log = data[['children', 'days_employed', 'dob_years', 'education_id',
       'family_status_id', 'gender', 'income_type', 'debt',
       'total_income', 'purpose_id']]

data_log.head(10)

In [ ]:
import warnings
warnings.filterwarnings('ignore')

data_dict_education = data[['education', 'education_id']]
data_dict_family_status = data[['family_status', 'family_status_id']]
data_dict_purpose = data[['purpose', 'purpose_id']]

data_dict_education = data_dict_education.drop_duplicates().reset_index(drop = True)
display(data_dict_education)
print('')

data_dict_family_status = data_dict_family_status.drop_duplicates().reset_index(drop = True)
display(data_dict_family_status)
print('')

data_dict_purpose = data_dict_purpose.drop_duplicates().reset_index(drop = True)
display(data_dict_purpose.sort_values(by ='purpose_id'))
print('')

In [ ]:
data['total_income_groups'] = pd.qcut(data['total_income'], 5)
data['total_income_groups'].value_counts()

### Вывод

Создадим следующие словари:

1. Словарь образования. Для его создания необходимо привести все значения "образование" в нижний регистр.
2. Словарь семейного положения. Он необходим, потому что семейное положение записывается длинными строками. Гораздо удобнее будет обращение по индексу/номеру.
3. Словарь целей кредита. Так же необходим по причине длинных записей, а так же разных записей обозначающих одну цель.

Значения столбца с доходом разобьем на 5 равных категорий при помощи метода qcut().

## 3. Ответы на вопросы

1) Есть ли зависимость между наличием детей и возвратом кредита в срок?

In [ ]:
data_children = data[['children', 'debt']]
print(data_children['children'].value_counts())
print('')

data_children['children'] = data_children['children'].replace((4,5,20), '>3')
data_children['children'] = data_children['children'].replace((-1), 0)

data_children = data_children.groupby('children').agg({'debt' : ['sum', 'count', 'mean']})

def percent(value):
    new_value = '{:.1%}'.format(value)
    return new_value
data_children['debt', 'mean'] = data_children['debt', 'mean'].apply(percent)
display(data_children)


### Вывод

Для начала посмотрим количество значений столбца "Дети". Ожидаемо, самые часто встречающиеся значения 0,1,2,3. Так же есть люди у которых 4 и 5 детей, но таких людей мало, поэтому объединим их в одну категорию "больше 3" детей. Ещё в данных присутствуют артефакты: -1 ребёнок и 20 детей. количесвто таких данных много меньше общего числа данных, поэтому можно от них просто избавится, на общую статистику это не повлияет. А можно предположить что -1 ребёнок это отсутствие детей, а 20 детей это "больше 3"

Избавившись от артефактов получаем результат, представим его в виде процента людей, имеющих задолженность по кредиту. Можно сделать вывод, что люди без детей исправнее платят по кредиту, потому что легче планировать свой бюджет. Тяжелее всего спланировать свои финансы приходится людям с 2 детьми.

2) Есть ли зависимость между семейным положением и возвратом кредита в срок?

In [ ]:
data_family = data[['family_status', 'debt']]

data_family = data_family.groupby('family_status').agg({'debt' : ['sum', 'count', 'mean']})

data_family['debt', 'mean'] = data_family['debt', 'mean'].apply(percent)
display(data_family)

### Вывод

Можно сделать вывод, что люди, которые состоят (или стостояли) в браке, исправнее платят по кредитам, чем люди, которые в браке никогда не состояли. Возможно это вызвано большей ответственностью, которую люди в браке несут по жизни.

3) Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

In [ ]:
data_income = data[['total_income_groups', 'debt']]

data_income = data_income.groupby('total_income_groups').agg({'debt' : ['sum', 'count', 'mean']})

data_income['debt', 'mean'] = data_income['debt', 'mean'].apply(percent)
display(data_income)

### Вывод

Исправнее всего платят люди с высокой (больше 200 тыс. руб.), потому что у таких людей достаточный доход для погашения кредитов в срок. На втором месте находятся люди из катгории самых невысоких зарплат (20 - 98 тыс. руб.), Такая зависимость может наблюдаться потому что люди с не высокой зарплатой привыкли жить, тщательно планируя свой бюджет. А серединная категория людей (от 98 до 214) имеет больший процент должников, возможно, потому что люди из этой категории не испытывают необходимости тщательно планировать свой бюджет, что приводит к нерациональным тратам и нехватке средств на погашение кредита.

4) Как разные цели кредита влияют на его возврат в срок?

In [ ]:
data_purpose = data[['purpose_id', 'debt']]

data_purpose = data_purpose.pivot_table(values = 'debt', index = 'purpose_id', aggfunc = 'mean')
data_purpose['debt'] = data_purpose['debt'].apply(percent) 
display(data_purpose)

### Вывод

Минимальный процент задолженностей наблюдается по ипотечным кредитам, а максимальный по автомобильным и кредитам на образование. Кредиты на организацию свадьбы занимают место посередине.

## 4. Общий вывод

В данной работе в качесвте исходных данных от банка была получена статистика о платежеспособности клиентов. 

В ходе работы данные были проанализированы и обработаны:
1. приведены к удобной для использования форме
2. удалены дубликаты
3. обработаны пропущенные значения
4. изменен тип данных, где необходимо
5. все данные категоризированы

В результате были получены зависимости платежеспособности клиентов от количества детей, семейного положения, общего дохода и цели кредита. Исправнее всего платят:
1. люди без детей - 7,5% должников, против 9,5% среди людей с двумя детьми
2. люди состоящие/состоявшие в браке ~7% должников, против ~9,5% среди людей, никогда не бывавшими в браке
3. люди с высокой зарплатой >214 тыс. руб. - 7% должников, против ~8,2% среди людей с зарплатой меньше
4. люди, взявшие ипотеку - 7,2% должников